In [1]:
import pandas as pd
from sqlalchemy import create_engine, text

# Remplacez ces valeurs par les informations de votre base de données
db_host = "localhost"
db_name = "olist"
db_user = "postgres"
db_password = "admin"

# Créer une connexion avec la base de données en utilisant sqlalchemy
engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")


In [3]:
# Lire le fichier CSV avec pandas
data = pd.read_csv("data/olist_sellers_dataset.csv", encoding="utf-8")
data.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [39]:
# # Ajouter les codes postaux manquants
# unique_zip_codes = data['seller_zip_code_prefix'].drop_duplicates()
# zip_codes_db = pd.read_sql("SELECT zip_code_prefix FROM olist_zipcode", engine)
# missing_zip_codes = unique_zip_codes.loc[~unique_zip_codes.isin(zip_codes_db['zip_code_prefix'])]
# missing_zip_codes = pd.DataFrame(missing_zip_codes, columns=['zip_code_prefix'])
# missing_zip_codes.to_sql('olist_zipcode', engine, if_exists='append', index=False)

# Ajouter les codes postaux manquants
unique_zip_codes = data['seller_zip_code_prefix'].drop_duplicates().to_frame()
unique_zip_codes.columns = ['zip_code_prefix']
unique_zip_codes['zip_code_prefix'] = unique_zip_codes['zip_code_prefix'].astype(str)

existing_zip_codes = pd.read_sql("SELECT zip_code_prefix FROM olist_zipcode", engine)
existing_zip_codes['zip_code_prefix'] = existing_zip_codes['zip_code_prefix'].astype(str)

# Fusionner les deux dataframes pour trouver les codes postaux manquants
missing_zip_codes = unique_zip_codes.merge(existing_zip_codes, on='zip_code_prefix', how='left', indicator=True)
missing_zip_codes = missing_zip_codes[missing_zip_codes['_merge'] == 'left_only'][['zip_code_prefix']]

# Ajouter les codes postaux manquants à la table olist_zipcode
missing_zip_codes.to_sql('olist_zipcode', engine, if_exists='append', index=False)


84

In [40]:
# Ajouter les états manquants
unique_states = data['seller_state'].drop_duplicates()
states_db = pd.read_sql("SELECT state_code FROM olist_state", engine)
missing_states = unique_states.loc[~unique_states.isin(states_db['state_code'])]
missing_states = pd.DataFrame(missing_states, columns=['state_code'])
missing_states.to_sql('olist_state', engine, if_exists='append', index=False)


0

In [41]:
# Ajouter les villes manquantes
unique_cities = data[['seller_city', 'seller_state']].drop_duplicates()
cities_db = pd.read_sql("SELECT city_name, state_code FROM olist_city", engine)
missing_cities = unique_cities.merge(cities_db, left_on=['seller_city', 'seller_state'], right_on=['city_name', 'state_code'], how='left', indicator=True)
missing_cities = missing_cities[missing_cities['_merge'] == 'left_only'][['seller_city', 'seller_state']]
missing_cities.columns = ['city_name', 'state_code'] # Renommer les colonnes pour correspondre à la table city
missing_cities.to_sql('olist_city', engine, if_exists='append', index=False)

0

In [42]:
# Maintenant, ajouter les relations manquantes entre les villes et les codes postaux
cities_db = pd.read_sql("SELECT city_id, city_name, state_code FROM olist_city", engine)
city_zip_codes = data[['seller_city', 'seller_state', 'seller_zip_code_prefix']].drop_duplicates()
city_zip_codes_joined = city_zip_codes.merge(cities_db, left_on=['seller_city', 'seller_state'], right_on=['city_name', 'state_code'])
city_zip_codes_joined = city_zip_codes_joined[['city_id', 'seller_zip_code_prefix']]
city_zip_codes_joined.columns = ['city_id', 'zip_code_prefix']  # Renommer les colonnes pour correspondre à la table city_zip_code

existing_city_zip_codes = pd.read_sql("SELECT city_id, zip_code_prefix FROM olist_cityzipcode", engine)

# Convertir les types de données pour qu'ils correspondent
city_zip_codes_joined['city_id'] = city_zip_codes_joined['city_id'].astype(int)
city_zip_codes_joined['zip_code_prefix'] = city_zip_codes_joined['zip_code_prefix'].astype(str)
existing_city_zip_codes['city_id'] = existing_city_zip_codes['city_id'].astype(int)
existing_city_zip_codes['zip_code_prefix'] = existing_city_zip_codes['zip_code_prefix'].astype(str)

city_zip_codes_to_insert = city_zip_codes_joined.merge(existing_city_zip_codes, on=['city_id', 'zip_code_prefix'], how='left', indicator=True)
city_zip_codes_to_insert = city_zip_codes_to_insert[city_zip_codes_to_insert['_merge'] == 'left_only'][['city_id', 'zip_code_prefix']]

city_zip_codes_to_insert.to_sql('olist_cityzipcode', engine, if_exists='append', index=False)


215

In [44]:
# Insérer les données des vendeurs dans la table Seller
sellers_data = data[['seller_id', 'seller_zip_code_prefix']].copy()
sellers_data.columns = ['seller_id', 'zip_code_prefix']  # Renommer la colonne pour correspondre au modèle
sellers_data.to_sql('olist_seller', engine, if_exists='append', index=False)


95